# TinyAleph: Kuramoto Oscillators and Synchronization

This notebook explores coupled oscillator dynamics:

- **Kuramoto Model**: Phase oscillators with mean-field coupling
- **Order Parameter**: Measuring collective synchronization
- **Phase Transitions**: From incoherence to synchronization
- **Stochastic Dynamics**: Adding noise and temperature

In [ ]:
import sys
sys.path.insert(0, '..')

from tinyaleph.physics.kuramoto import KuramotoOscillator, KuramotoField
from tinyaleph.physics.stochastic import StochasticKuramotoOscillator, ThermalKuramotoField
from tinyaleph.physics.lyapunov import LyapunovExponent, StabilityClass
from tinyaleph.core.constants import PHI, CRITICAL_COUPLING
import math
import numpy as np
import matplotlib.pyplot as plt

print(f"Critical coupling K_c = π/2 ≈ {CRITICAL_COUPLING:.4f}")

## 1. The Kuramoto Model

The Kuramoto model describes coupled phase oscillators:

$$\frac{d\theta_i}{dt} = \omega_i + \frac{K}{N} \sum_{j=1}^{N} \sin(\theta_j - \theta_i)$$

Where:
- $\theta_i$ is the phase of oscillator i
- $\omega_i$ is its natural frequency
- $K$ is the coupling strength

In [ ]:
# Single oscillator
osc = KuramotoOscillator(omega=1.0, phase=0.0)
print(f"Initial: phase={osc.phase:.4f}, frequency={osc.omega}")

# Evolve without coupling
for _ in range(10):
    osc.step(force=0, dt=0.1)
    
print(f"After 1s: phase={osc.phase:.4f} rad = {math.degrees(osc.phase) % 360:.1f}°")

## 2. Order Parameter

The **order parameter** measures collective synchronization:

$$r \cdot e^{i\psi} = \frac{1}{N} \sum_{j=1}^{N} e^{i\theta_j}$$

- $r = 0$: Incoherent (random phases)
- $r = 1$: Fully synchronized
- $0 < r < 1$: Partial synchronization

In [ ]:
# Create oscillator field
field = KuramotoField(num_oscillators=100, coupling=2.0, omega_spread=0.5)

print(f"N = {field.N} oscillators")
print(f"K = {field.K} coupling")
print(f"Initial order parameter r = {field.order_parameter():.4f}")

In [ ]:
# Evolve and track synchronization
steps = 500
dt = 0.1
r_history = []

for step in range(steps):
    field.step(dt=dt)
    r_history.append(field.order_parameter())

plt.figure(figsize=(10, 5))
plt.plot(np.arange(steps) * dt, r_history, 'b-', linewidth=1.5)
plt.axhline(y=1/PHI, color='r', linestyle='--', label=f'1/φ = {1/PHI:.3f}')
plt.xlabel('Time', fontsize=12)
plt.ylabel('Order Parameter r', fontsize=12)
plt.title('Kuramoto Synchronization', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim([0, 1])
plt.show()

print(f"Final order parameter: r = {r_history[-1]:.4f}")

## 3. Phase Transition

The Kuramoto model exhibits a phase transition at critical coupling $K_c$.

In [ ]:
# Sweep coupling strength
coupling_values = np.linspace(0.5, 4.0, 15)
final_r_values = []

for K in coupling_values:
    field = KuramotoField(num_oscillators=100, coupling=K, omega_spread=0.5, random_seed=42)
    
    # Equilibrate
    for _ in range(300):
        field.step(dt=0.1)
    
    # Average over last steps
    r_avg = 0
    for _ in range(50):
        field.step(dt=0.1)
        r_avg += field.order_parameter()
    r_avg /= 50
    
    final_r_values.append(r_avg)

plt.figure(figsize=(10, 6))
plt.plot(coupling_values, final_r_values, 'bo-', linewidth=2, markersize=8)
plt.axvline(x=CRITICAL_COUPLING, color='r', linestyle='--', label=f'K_c = π/2 ≈ {CRITICAL_COUPLING:.2f}')
plt.xlabel('Coupling Strength K', fontsize=12)
plt.ylabel('Order Parameter r', fontsize=12)
plt.title('Kuramoto Phase Transition', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 4. Phase Distribution

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, K in zip(axes, [0.5, 1.5, 3.0]):
    field = KuramotoField(num_oscillators=100, coupling=K, random_seed=42)
    for _ in range(500):
        field.step(dt=0.1)
    
    phases = [p % (2*math.pi) for p in field.phases]
    ax.hist(phases, bins=30, density=True, alpha=0.7, color='steelblue')
    ax.set_xlabel('Phase (radians)')
    ax.set_ylabel('Density')
    ax.set_title(f'K = {K}, r = {field.order_parameter():.2f}')
    ax.set_xlim([0, 2*math.pi])

plt.tight_layout()
plt.show()

## 5. Stochastic Kuramoto (with noise)

In [ ]:
# Stochastic oscillator
stoch_osc = StochasticKuramotoOscillator(omega=1.0, phase=0.0, noise_strength=0.5)

phases = []
for _ in range(500):
    stoch_osc.step(force=0, dt=0.1)
    phases.append(stoch_osc.phase)

plt.figure(figsize=(10, 4))
plt.plot(phases, 'b-', alpha=0.7)
plt.xlabel('Step')
plt.ylabel('Phase')
plt.title('Stochastic Oscillator with Noise')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Thermal field - temperature affects synchronization
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, T in zip(axes, [0.1, 0.5, 1.0]):
    field = ThermalKuramotoField(num_oscillators=50, coupling=2.0, temperature=T)
    r_history = []
    
    for _ in range(300):
        field.step(dt=0.1)
        r_history.append(field.order_parameter())
    
    ax.plot(r_history, 'b-', alpha=0.7)
    ax.set_xlabel('Step')
    ax.set_ylabel('Order Parameter r')
    ax.set_title(f'Temperature T = {T}')
    ax.set_ylim([0, 1])
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Lyapunov Stability Analysis

In [ ]:
# Track Lyapunov exponent
field = KuramotoField(num_oscillators=50, coupling=2.0)
lyap = LyapunovExponent(window_size=50)

for _ in range(200):
    field.step(dt=0.1)
    lyap.observe(field.order_parameter())

print(f"Lyapunov exponent: λ = {lyap.current_exponent():.4f}")
print(f"Stability class: {lyap.classify()}")

## Summary

- **Kuramoto model** describes coupled phase oscillators
- **Order parameter r** measures synchronization (0=incoherent, 1=synchronized)
- **Phase transition** occurs at critical coupling K_c ≈ π/2
- **Temperature/noise** disrupts synchronization
- **Lyapunov exponent** characterizes stability